<a href="https://colab.research.google.com/github/Shrutika-TechSavvy/Google-Colab-Codes/blob/main/Matrix_Multiplication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%writefile matrix_mul_colab.cu
#include <stdio.h>
#include <cuda.h>

#define N 3

__global__ void matMul(int *A, int *B, int *C, int n) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < n && col < n) {
        int sum = 0;
        for (int k = 0; k < n; ++k)
            sum += A[row * n + k] * B[k * n + col];
        C[row * n + col] = sum;
    }
}

int main() {
    int size = N*N*sizeof(int);
    int h_A[N*N], h_B[N*N], h_C[N*N];

    // Initialize matrices
    for (int i = 0; i < N*N; i++) {
        h_A[i] = i + 1;
        h_B[i] = (i + 1) * 2;
    }

    int *d_A, *d_B, *d_C;
    cudaMalloc(&d_A, size);
    cudaMalloc(&d_B, size);
    cudaMalloc(&d_C, size);
    cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice);

    dim3 threads(N, N);
    dim3 blocks(1, 1);
    matMul<<<blocks, threads>>>(d_A, d_B, d_C, N);
    cudaDeviceSynchronize();

    cudaMemcpy(h_C, d_C, size, cudaMemcpyDeviceToHost);

    printf("Result matrix C:\n");
    for (int i = 0; i < N*N; ++i) {
        printf("%d ", h_C[i]);
        if ((i+1) % N == 0) printf("\n");
    }

    cudaFree(d_A); cudaFree(d_B); cudaFree(d_C);
    return 0;
}


Writing matrix_mul_colab.cu


In [2]:
!nvcc -arch=sm_75 matrix_mul_colab.cu -o matrix_mul_colab
!./matrix_mul_colab


Result matrix C:
60 72 84 
132 162 192 
204 252 300 


In [ ]:
!./matmul